In [38]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy

In [39]:
input_path = '/home/werewolf97/imtech/pro/fulldata/'

In [40]:
input_path = Path(input_path)

In [41]:
text_data = []

In [42]:
input_path

PosixPath('/home/werewolf97/imtech/pro/fulldata')

In [43]:
converted_path = Path.joinpath(input_path.parent, "converted_files")
if not os.path.exists(converted_path):
    os.mkdir(converted_path)
#print(type(self.input_path.as_posix()))
files = input_path.rglob(f"*.pdf")
#print("Files found: {0}".format(files))
exts = [".pdf", ".txt"]

In [44]:
for file in input_path.rglob('*'):
#     path = dirc+file
    with open(file, "r", encoding="utf-8") as f:
        content = f.read().replace('\n', ' ')
    if content is not None:
        data = [f'{file.parent.name}_{file.name}', content]
        text_data.append(data)
text_corpus = pd.DataFrame(text_data, columns=['resource_id', 'description'])

In [45]:
corpus = text_corpus

In [46]:
corpus['full_text'] = ""
corpus['full_text'] = corpus['description']

In [47]:
corpus

,resource_id,description,full_text
0,fulldata_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...
1,fulldata_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...
2,fulldata_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar..."
3,fulldata_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...
4,fulldata_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...
...,...,...,...
62,fulldata_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...
63,fulldata_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...
64,fulldata_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...
65,fulldata_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...


In [48]:
def strip_html_tags(text):
#     """Strips the HTML Tags from the text"""
    try:
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text()
    except:
        stripped_text = text
    return stripped_text

In [49]:
def remove_accented_chars(text):
#         """Remove Non UTF-8 Characters from text"""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [50]:
def expand_contractions(text, contraction_mapping=contractions.CONTRACTION_MAP):
#         """Replace contractions in string of text"""
        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                          flags=re.IGNORECASE | re.DOTALL)

        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match) \
                if contraction_mapping.get(match) \
                else contraction_mapping.get(match.lower())
            expanded_contraction = first_char + expanded_contraction[1:]
            return expanded_contraction

        try:
            expanded_text = contractions_pattern.sub(expand_match, text)
            expanded_text = re.sub("'", "", expanded_text)
        except:
            expanded_text = text
        return expanded_text

In [51]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [52]:
def lemmatize_text(text):
#         """Lemmatize the text using the WordNetLemmatizer"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = tokenizer.tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return text

In [53]:
def remove_special_characters(text, remove_digits):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [54]:
def remove_stopwords( text):
#         """Remove Stopwords defined in the stopwords corpus"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    stopword_list = nltk.corpus.stopwords.words('english')
    stopword_list.remove('no')
    stopword_list.remove('not')
    stopword_list.append('cid')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    tokens = filtered_tokens
    stopword_list = google_list.stopwords
    stopword_list.append('cid')
#     filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [55]:
def normalize_corpus(corpus):
    normalized_corpus = []
    counter = 0
    for i in range(len(corpus)):
        doc = corpus['description'][i]
        counter += 1
        doc = strip_html_tags(doc)
        doc = remove_accented_chars(doc)
        doc = expand_contractions(doc)
        doc = doc.lower()
        doc = re.sub(r'[\r|\n|\r\n]+', ' ', doc)
        doc = lemmatize_text(doc)
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        doc = remove_special_characters(doc, remove_digits= True)
        doc = re.sub(' +', ' ', doc)
        doc = remove_stopwords(doc)
        normalized_corpus.append(doc)
    return normalized_corpus

In [56]:
corpus['clean_text'] = normalize_corpus(corpus)

In [57]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [58]:
corpus['tokens'] = get_tokens(corpus['clean_text'])

In [59]:
import en_core_web_sm
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = en_core_web_sm.load()

In [60]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=20):
        """Print top number keywords"""
        keywords = []
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
#             print(key + ' - ' + str(value))
            keywords.append(key)
            if i > number:
                break
        return keywords
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [61]:
obj =  TextRank4Keyword()

In [62]:
keywords = []
for i in range(len(corpus)): 
    text = corpus['clean_text'][i]
    obj.analyze(text)
    a = obj.get_keywords(500)
    #     b = corpus['tokens'][i]
    #     l = []
    #     for i in a:
    #         if i in b:
    #             l.append(i)
    keywords.append(a)

In [63]:
corpus['keywords'] = keywords

In [64]:
corpus

,resource_id,description,full_text,clean_text,tokens,keywords
0,fulldata_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,connectivity frank kammer hanjo aubig chapter ...,"[connectivity, frank, kammer, hanjo, aubig, ch...","[graph, vertex, g, edge, algorithm, v, connect..."
1,fulldata_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,network science web mandate structural analysi...,"[network, science, web, mandate, structural, a...","[graph, node, centrality, game, network, v, me..."
2,fulldata_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality index dirk kosch utzki katharina an...,"[centrality, index, dirk, kosch, utzki, kathar...","[centrality, vertex, graph, index, v, network,..."
3,fulldata_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,social learning network module network science...,"[social, learning, network, module, network, s...","[belief, learning, agent, network, control, bi..."
4,fulldata_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,chapter matching market seen number way thinki...,"[chapter, matching, market, seen, number, way,...","[price, graph, buyer, matching, valuation, sel..."
...,...,...,...,...,...,...
62,fulldata_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,chapter link analysis web search searching web...,"[chapter, link, analysis, web, search, searchi...","[page, search, pagerank, web, authority, link,..."
63,fulldata_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,network comparison michael baur marc benkert f...,"[network, comparison, michael, baur, marc, ben...","[graph, g, vertex, v, algorithm, distance, f, ..."
64,fulldata_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,introduction ulrik brandes thomas erlebach man...,"[introduction, ulrik, brandes, thomas, erlebac...","[network, analysis, www, document, chapter, bo..."
65,fulldata_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley value recap cooperating agent agent wi...,"[shapley, value, recap, cooperating, agent, ag...","[value, node, coalition, network, j, shapley, ..."


In [65]:
topics=[]
for i in range(len(corpus)):
    l = corpus['tokens'][i]
    k = corpus['keywords'][i]
    dic = {}
    for j in l:
        if j not in k:
            continue
        if j in dic.keys():
            dic[j]+=1
        else:
            dic[j]=1
    sort = sorted(dic, key=dic.get)
    sort.reverse()
    l = []
    for j in range(10):
        l.append(sort[j])
    topics.append(l)

In [66]:
topics = pd.DataFrame(topics)

In [67]:
topics['resource_id']= corpus['resource_id']

In [68]:
topics = topics[['resource_id',0,1,2,3,4,5,6,7,8,9]]

In [69]:
topics

,resource_id,0,1,2,3,4,5,6,7,8,9
0,fulldata_LNch7.txt,vertex,graph,edge,minimum,cut,set,algorithm,tree,component,connectivity
1,fulldata_Mandate2.txt,graph,node,centrality,measure,element,network,game,utility,closeness,vitality
2,fulldata_LNch3.txt,centrality,vertex,graph,index,path,edge,network,value,section,measure
3,fulldata_SocialLearningL2.txt,belief,learning,agent,social,network,control,signal,model,influence,bias
4,fulldata_NCMch10.txt,matching,price,buyer,set,seller,valuation,house,node,graph,auction
...,...,...,...,...,...,...,...,...,...,...,...
62,fulldata_NCMch14.txt,pagerank,page,node,link,value,authority,update,web,hub,search
63,fulldata_LNch12.txt,graph,vertex,algorithm,let,isomorphism,partition,distance,order,cell,number
64,fulldata_LNch1.txt,network,analysis,document,chapter,www,group,com,web,graph,actor
65,fulldata_NetworkCENTRALITYL2.txt,node,value,fsv,network,coalition,link,shapley,fringe,degree,agent


In [70]:
topics.to_csv("top_10_textrank.csv")